In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:

import time
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [ ]:
import mlflow

In [ ]:

import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate

In [ ]:
df=pd.read_csv('Final_merged_purchased.csv')

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)


In [ ]:
df.columns

Index(['Unnamed: 0', 'event_time', 'event_type', 'product_id', 'category_id',
       'brand', 'price', 'user_id', 'user_session', 'product_count',
       'order_id'],
      dtype='object')

In [ ]:
print("number of unique customer : ",df["user_id"].nunique())
print("number of unique product : ",df["product_id"].nunique())

number of unique customer :  52523
number of unique product :  33167


In [ ]:
def remove_strip(x:pd.Series)->pd.Series:
    x=x.replace("-","",regex=True)
    return x

df['customer_id']=remove_strip(df['user_id'])

In [ ]:
# just to make sure that all Nan containing rows are deleted..
print("No of Nan values in our dataframe : ", sum(df.isnull().any()))

No of Nan values in our dataframe :  0


In [ ]:
dup_bool = df.duplicated(['user_id','product_id','product_count'])
dups = sum(dup_bool) # by considering all columns..( including timestamp)
print("There are {} duplicate rating entries in the data..".format(dups))

There are 17857 duplicate rating entries in the data..


In [ ]:
#df.drop_duplicates()

In [ ]:
df.shape

(567514, 12)

In [ ]:
df.columns

Index(['Unnamed: 0', 'event_time', 'event_type', 'product_id', 'category_id',
       'brand', 'price', 'user_id', 'user_session', 'product_count',
       'order_id', 'customer_id'],
      dtype='object')

In [ ]:
df=df[['event_time', 'event_type', 'product_id', 'category_id',
       'brand', 'price', 'user_id', 'user_session', 'product_count',
       'order_id', 'customer_id']]

In [ ]:
#Split the training and test data in the ratio 70:30
train_data, test_data = train_test_split(df, test_size = 0.3, random_state=0)
print("data train sample : ")

print("===="*20)
print("train data shape : ",train_data.shape)
print("test data shape : ",test_data.shape)

data train sample : 
train data shape :  (397259, 11)
test data shape :  (170255, 11)


In [ ]:
train_data.columns

Index(['event_time', 'event_type', 'product_id', 'category_id', 'brand',
       'price', 'user_id', 'user_session', 'product_count', 'order_id',
       'customer_id'],
      dtype='object')

In [ ]:
reader=Reader(rating_scale=(0,10))
data=Dataset.load_from_df(df[["user_id","product_id","product_count"]],reader)

In [ ]:
from surprise.model_selection import train_test_split
trainset,testset=train_test_split(data,test_size=0.25,random_state=121)

In [ ]:
# We can now use the algorithm that yields the best rmse:
SVDpp = SVDpp(n_factors=20,n_epochs=50,lr_all=0.0003,reg_all=0.03)
SVDpp.fit(trainset)
# Predict ratings for all pairs (i,j) that are NOT in the training set.
SVDpp_test = SVDpp.test(testset)

In [ ]:
len(SVDpp_test)

141879

In [ ]:
from surprise import accuracy

In [ ]:
RMSE=accuracy.rmse(SVDpp_test,verbose=False)
MAE=accuracy.mae(SVDpp_test,verbose=False)

In [ ]:
MAE

In [ ]:
RMSE

In [ ]:


print("RMSE : ",RMSE)
print("MAE  : ",MAE)

RMSE :  0.056768745546791795
MAE  :  0.03915786816856314


In [ ]:
# # We can now use the algorithm that yields the best rmse:
# SVDpp = SVDpp(n_factors=20,n_epochs=50,lr_all=0.0003,reg_all=0.03)
# SVDpp.fit(trainset)
# # Predict ratings for all pairs (i,j) that are NOT in the training set.
# SVDpp_test = SVDpp.test(testset)
# RMSE :  0.05716347327509343
# MAE  :  0.03933804302943817